# ADS2 - Assignment 1 - Data Handling and Processing with PySpark

**STUDENT NAME -** Friday Odeh Ovbiroro

**STUDENT ID -** 22034665

In this assignment, you will be analysing the popularity of films and TV shows on the streaming platform, Netflix. Using your knowledge of PySpark DataFrames and Spark SQL, you will produce a number of "downstream" data products to analyse trends in global streaming habits.

Download the dataset from this [Kaggle](https://www.kaggle.com/dhruvildave/netflix-top-10-tv-shows-and-films) page. A copy of the `all_weeks_countries.csv` file is also available on the canvas page for this assignment.

Your task is to load in the data and produce a number of "downstream" data products and plots as described below.

The PySpark installation and setup is provided below for conveinience.

IMPORTANT: DO NOT EDIT OR REMOVE THE COMMENT TAGS IN THE CODE CELLS

In [ ]:
# CodeGrade Tag Init1

# Apache Spark uses Java, so first we must install that
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
# CodeGrade Tag Init2
# Mount Google Drive and unpack Spark
from google.colab import drive
drive.mount('/content/drive')
!tar xzf /content/drive/MyDrive/spark-3.3.0-bin-hadoop3.tgz

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# CodeGrade Tag Init3
# Set up environment variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.0-bin-hadoop3"

In [ ]:
# CodeGrade Tag Init4
# Install findspark, which helps python locate the pyspark module files
!pip install -q findspark
import findspark
findspark.init()

In [ ]:
# Finally, we initialse a "SparkSession", which handles the computations
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [ ]:
# Load the all_weeks_countries.csv into your Colab Notebook as a DataFrame.
netflixcsvpath = '/content/drive/MyDrive/all-weeks-countries.csv'

# Data is loaded with header: True and an inferred schema
netflixDF = (spark
           .read
           .option('header', 'True')
           .option('inferSchema', 'True')
           .csv(netflixcsvpath)
          )

In [ ]:
# pyspark.sql.functions countains all the transformations and actions you will
# need
from pyspark.sql import functions as F

# Exercise 1 - Data Preparation


1.   Create two separate DataFrames for Films and TV.
2.   For the Films data, drop the column containing the season names.
3.   For the TV data, replace any null values in the season name column with the show name.

In [ ]:
# CodeGrade Tag Ex1a

### Display the table and its schema

netflixDF.show() # displaying the table
netflixDF.printSchema() # displaying the schema

+------------+------------+-------------------+--------+-----------+--------------------+--------------------+--------------------------+
|country_name|country_iso2|               week|category|weekly_rank|          show_title|        season_title|cumulative_weeks_in_top_10|
+------------+------------+-------------------+--------+-----------+--------------------+--------------------+--------------------------+
|   Argentina|          AR|2022-08-21 00:00:00|   Films|          1|      Look Both Ways|                null|                         1|
|   Argentina|          AR|2022-08-21 00:00:00|   Films|          2|           Day Shift|                null|                         2|
|   Argentina|          AR|2022-08-21 00:00:00|   Films|          3|Bank Robbers: The...|                null|                         2|
|   Argentina|          AR|2022-08-21 00:00:00|   Films|          4|   The Next 365 Days|                null|                         1|
|   Argentina|          AR|2022-08

In [ ]:
# CodeGrade Tag Ex1b

### Seperate the data into two DataFrames for Films and TV
### Call the dataframes tvDF and filmsDF

filmsDF = netflixDF.filter(netflixDF['category'] == 'Films')
filmsDF.show()

tvDF = netflixDF.filter(netflixDF['category'] == 'TV')
tvDF.show()


+------------+------------+-------------------+--------+-----------+--------------------+------------+--------------------------+
|country_name|country_iso2|               week|category|weekly_rank|          show_title|season_title|cumulative_weeks_in_top_10|
+------------+------------+-------------------+--------+-----------+--------------------+------------+--------------------------+
|   Argentina|          AR|2022-08-21 00:00:00|   Films|          1|      Look Both Ways|        null|                         1|
|   Argentina|          AR|2022-08-21 00:00:00|   Films|          2|           Day Shift|        null|                         2|
|   Argentina|          AR|2022-08-21 00:00:00|   Films|          3|Bank Robbers: The...|        null|                         2|
|   Argentina|          AR|2022-08-21 00:00:00|   Films|          4|   The Next 365 Days|        null|                         1|
|   Argentina|          AR|2022-08-21 00:00:00|   Films|          5|The Angry Birds M...| 

In [ ]:
# CodeGrade Tag Ex1c

### Drop the 'season_title' column from the Films DataFrame, display the table

filmsDF_1 = filmsDF.drop('season_title')
filmsDF_1.show()

+------------+------------+-------------------+--------+-----------+--------------------+--------------------------+
|country_name|country_iso2|               week|category|weekly_rank|          show_title|cumulative_weeks_in_top_10|
+------------+------------+-------------------+--------+-----------+--------------------+--------------------------+
|   Argentina|          AR|2022-08-21 00:00:00|   Films|          1|      Look Both Ways|                         1|
|   Argentina|          AR|2022-08-21 00:00:00|   Films|          2|           Day Shift|                         2|
|   Argentina|          AR|2022-08-21 00:00:00|   Films|          3|Bank Robbers: The...|                         2|
|   Argentina|          AR|2022-08-21 00:00:00|   Films|          4|   The Next 365 Days|                         1|
|   Argentina|          AR|2022-08-21 00:00:00|   Films|          5|The Angry Birds M...|                         1|
|   Argentina|          AR|2022-08-21 00:00:00|   Films|        

In [ ]:
# CodeGrade Tag Ex1d

### Use the F.isnull function to create a column showing where there are null
### values in the 'season_title' column. Replace the null values with the
### corresponding value from the 'show_title' column, then replace the
### 'season_title' column in the tvDF DataFrame.


ex1 = netflixDF.withColumn('null_season', F.isnull(netflixDF['season_title']))
ex1a = ex1.withColumn('season_title', F.when(ex1['null_season'] =='true', ex1['show_title']).otherwise(ex1['season_title']))
ex1a.show()


df= tvDF.withColumn('null_season', F.isnull(tvDF['season_title']))
tvDF = df.withColumn('season_title', F.when(df['null_season'] =='true', df['show_title']).otherwise(df['season_title']))
tvDF.show()


+------------+------------+-------------------+--------+-----------+--------------------+--------------------+--------------------------+-----------+
|country_name|country_iso2|               week|category|weekly_rank|          show_title|        season_title|cumulative_weeks_in_top_10|null_season|
+------------+------------+-------------------+--------+-----------+--------------------+--------------------+--------------------------+-----------+
|   Argentina|          AR|2022-08-21 00:00:00|   Films|          1|      Look Both Ways|      Look Both Ways|                         1|       true|
|   Argentina|          AR|2022-08-21 00:00:00|   Films|          2|           Day Shift|           Day Shift|                         2|       true|
|   Argentina|          AR|2022-08-21 00:00:00|   Films|          3|Bank Robbers: The...|Bank Robbers: The...|                         2|       true|
|   Argentina|          AR|2022-08-21 00:00:00|   Films|          4|   The Next 365 Days|   The Next

# Exercise 2

Making use of the "groupBy" and "where" methods, find the number of weeks the show "Stranger Things" was in the Top 10 for the United Kingdom across all seasons. Store your result in a variable named "STWeeks"


In [ ]:
# CodeGrade Tag Ex2
### Use the "where" method to create a new dataframe containing the data for
### the show Stranger Things in the Uniter Kingdom. Call this dataframe STDF.

STDF = netflixDF.where((netflixDF['show_title'] == 'Stranger Things') & (netflixDF['country_name'] == 'United Kingdom'))


### Using "groupBy" method and "F.count_distinct" function, find the total number of weeks
### Stranger Things spent in the top 10 of the UK, across all seasons. Show the
### result.

STWeeks = STDF.groupBy('show_title').agg(F.count_distinct('cumulative_weeks_in_top_10'))
STWeeks.show()

+---------------+---------------------------------+
|     show_title|count(cumulative_weeks_in_top_10)|
+---------------+---------------------------------+
|Stranger Things|                               13|
+---------------+---------------------------------+



# Exercise 3

Produce a dataframe containing only the Top 25 TV seasons in the UK, based on the number of weeks they spent in the Top 10.

In [ ]:
# CodeGrade Tag Ex3
### Produce a dataframe containing the top 25 seasons by number of weeks in the
### top 10 of the United Kingdom, sorted by number of weeks. Store the dataframe
### in a variable called Top25


# Filtering TV DataFrame for United Kingdom
ft = tvDF.filter(tvDF['country_name'] == 'United Kingdom')
Top25 = ft.groupBy('season_title').agg(F.max('cumulative_weeks_in_top_10').alias('max_weeks')).sort(F.desc('max_weeks')).limit(25)
Top25.show()

+--------------------+---------+
|        season_title|max_weeks|
+--------------------+---------+
|   Stranger Things 4|       13|
|     Ozark: Season 4|       12|
|   Stranger Things 3|       12|
|   Stranger Things 2|       11|
|Squid Game: Season 1|       10|
|     Stranger Things|       10|
|Better Call Saul:...|       10|
|Bridgerton: Season 2|        9|
|Maid: Limited Series|        9|
|PAW Patrol: Season 6|        9|
|Inventing Anna: L...|        8|
| Money Heist: Part 5|        7|
|  Sex/Life: Season 1|        7|
|PIECES OF HER: Se...|        7|
|Clickbait: Limite...|        6|
|Sex Education: Se...|        6|
|Virgin River: Sea...|        6|
|Young Sheldon: : ...|        6|
|Young Sheldon: : ...|        6|
|The Lincoln Lawye...|        6|
+--------------------+---------+
only showing top 20 rows



# Exercise 4

For the show "Young Sheldon", find the country where each season spent the most time in the Top 10.

In [ ]:
# CodeGrade Tag Ex4
### For each season of the show "Young Sheldon" find the countries where it spent
### the most time in the Top 10

young = netflixDF.filter(netflixDF['show_title'] == 'Young Sheldon')
Ex4 = young.groupBy("country_name", "season_title").agg(F.max("cumulative_weeks_in_top_10").alias("max_weeks_in_top_10")).sort(F.desc('max_weeks_in_top_10'))
Ex4a = Ex4.groupBy('season_title').agg(F.max('max_weeks_in_top_10').alias('max_weeks_in_top_10'))
young_answer = Ex4a.join(Ex4, on=['season_title', 'max_weeks_in_top_10'], how='inner').sort(F.desc('max_weeks_in_top_10'))
young_answer.show(truncate=False)


+-------------------------+-------------------+--------------+
|season_title             |max_weeks_in_top_10|country_name  |
+-------------------------+-------------------+--------------+
|Young Sheldon: : Season 2|6                  |United Kingdom|
|Young Sheldon: : Season 1|6                  |Canada        |
|Young Sheldon: : Season 1|6                  |United Kingdom|
|Young Sheldon: : Season 1|6                  |Ireland       |
|Young Sheldon: : Season 3|4                  |United Kingdom|
|Young Sheldon: Season 4  |1                  |New Zealand   |
|Young Sheldon: Season 4  |1                  |Australia     |
+-------------------------+-------------------+--------------+



# Exercise 5

For each country, find the film that spent the most time in the Top 10.



In [ ]:
# CodeGrade Tag Ex5
### For each country, find the film that spent the most time in the Top 10
### Display the results in a Dataframe ordered by country name.


Ex5 = filmsDF.groupBy( "country_name", "show_title",).agg(
    F.max("cumulative_weeks_in_top_10").alias("max_weeks_in_top_10")).sort(F.desc("max_weeks_in_top_10"))
Ex5.show()


+------------+--------------------+-------------------+
|country_name|          show_title|max_weeks_in_top_10|
+------------+--------------------+-------------------+
|     Ukraine|          Red Notice|                 39|
|      Serbia|          Red Notice|                 23|
|     Vietnam|          Red Notice|                 22|
|  Bangladesh|        Sooryavanshi|                 21|
|     Ukraine|     Army of Thieves|                 21|
|    Pakistan|        Sooryavanshi|                 20|
|     Estonia|          Red Notice|                 20|
|      Taiwan|         Man in Love|                 20|
|       India|        Sooryavanshi|                 20|
|    Pakistan|     Haseen Dillruba|                 20|
|     Ukraine|       Don't Look Up|                 19|
|     Nigeria|Omo Ghetto: the Saga|                 19|
|     Hungary|          Red Notice|                 18|
|     Bolivia|               Shrek|                 18|
|  Bangladesh|     Haseen Dillruba|             

# Exercise 6

Calculate the number of weeks each film spent at the number 1 spot of each country's Top 10 list. Then find the films that spent the most time in the number 1 spot for each country.

In [ ]:
# CodeGrade Tag Ex6a

### Create a column using the F.when function to calculate the number of weeks a
### films spens in the number 1 spot of the Top 10. Use the .otherwise method to
### set rows with no number 1 spots to zero. Use the .alias metod to call this
### column "weeks_at_1"

Ex6a = filmsDF.withColumn("weeks_at_1", F.when(filmsDF["weekly_rank"] == 1, 1).otherwise(0).alias("weeks_at_1"))
Ex6a.show()

Ex6b= Ex6a.groupBy("country_name", "category").agg(
    F.max("cumulative_weeks_in_top_10").alias("max_weeks_in_top_10"),
    F.sum("weeks_at_1").alias("cumulative_weeks_at_1"))
Ex6b .show(truncate=False)


+------------+------------+-------------------+--------+-----------+--------------------+------------+--------------------------+----------+
|country_name|country_iso2|               week|category|weekly_rank|          show_title|season_title|cumulative_weeks_in_top_10|weeks_at_1|
+------------+------------+-------------------+--------+-----------+--------------------+------------+--------------------------+----------+
|   Argentina|          AR|2022-08-21 00:00:00|   Films|          1|      Look Both Ways|        null|                         1|         1|
|   Argentina|          AR|2022-08-21 00:00:00|   Films|          2|           Day Shift|        null|                         2|         0|
|   Argentina|          AR|2022-08-21 00:00:00|   Films|          3|Bank Robbers: The...|        null|                         2|         0|
|   Argentina|          AR|2022-08-21 00:00:00|   Films|          4|   The Next 365 Days|        null|                         1|         0|
|   Argentina

In [ ]:
# CodeGrade Tag Ex6b

### Group by country name and sow title, and use the .agg method and your new
### column to find the number of weeks each film spent in the top spot for each
### country.


ab= Ex6a.groupBy('country_name', 'show_title')
vf = ab.agg(F.sum('weeks_at_1').alias('total_weeks_at_1'))
df_6b= vf.sort('total_weeks_at_1', ascending=False)
df_6b.show()



+------------+-------------------+----------------+
|country_name|         show_title|total_weeks_at_1|
+------------+-------------------+----------------+
|      Russia|      Don't Look Up|               6|
|   Hong Kong|      Don't Look Up|               6|
|      Taiwan|      Don't Look Up|               5|
|      Taiwan|Gangubai Kathiawadi|               5|
|     Vietnam|         Red Notice|               5|
|      Israel|The Tinder Swindler|               4|
|     Estonia|      Don't Look Up|               4|
|  Bangladesh|    Haseen Dillruba|               4|
|     Ukraine|The Tinder Swindler|               4|
|     Vietnam|      Don't Look Up|               4|
|     Croatia|The Tinder Swindler|               4|
|    Bulgaria|         Red Notice|               4|
|       India|    Haseen Dillruba|               4|
|      Jordan|         Red Notice|               4|
|      Russia|The Tinder Swindler|               4|
|    Slovenia|      Don't Look Up|               4|
|    Pakista

In [ ]:
# CodeGrade Tag Ex6c

### Produce a dataframe grouped by country name that contains the show title and
### number of weeks at the number 1 spot of the top performing film in each
### country.

Ex6c = Ex6a.groupBy('country_name', 'show_title')\
       .agg(F.sum('weeks_at_1').alias('weeks_at_1'))\
       .orderBy('weeks_at_1', ascending=False)\
       .groupBy('country_name')\
       .agg(F.first('show_title'), F.max('weeks_at_1'))


Ex6c.show()

+------------------+-------------------+---------------+
|      country_name|  first(show_title)|max(weeks_at_1)|
+------------------+-------------------+---------------+
|         Argentina|      Don't Look Up|              3|
|         Australia|   The Adam Project|              3|
|           Austria|The Tinder Swindler|              4|
|           Bahamas|             Fatale|              2|
|           Bahrain|         The Guilty|              3|
|        Bangladesh|    Haseen Dillruba|              4|
|           Belgium|The Tinder Swindler|              3|
|           Bolivia|             Hustle|              3|
|            Brazil|       The Gray Man|              2|
|          Bulgaria|      Don't Look Up|              4|
|            Canada|   The Adam Project|              3|
|             Chile|      Don't Look Up|              3|
|          Colombia|         Red Notice|              3|
|        Costa Rica|         Red Notice|              3|
|           Croatia|The Tinder 